## This codes depicts Fine tuning of T5 model for question genration task

Framework: Pytorch



Dataset: squad dataset

Input: Context

Output: Genrated Question

Accelerated Environment required

Hugging Face account required

In [ ]:
#installing
!pip install datasets evaluate transformers rouge-score nltk
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 20.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=dbbe0e2b29c1da9f7e6b4c3325d5cf0c01c06b570682525c1b68be955af3586f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled

In [ ]:
#Import Necessary Statements
import transformers
from transformers.utils import send_example_telemetry
import datasets
from datasets import load_dataset
from evaluate import load
import random
import pandas as pd
import nltk
nltk.download('punkt')
import numpy as np
from IPython.display import display, HTML
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#connecting to Huggingface
!apt install git-lfs
from huggingface_hub import notebook_login

#hungingface login with Secret key
# While setting permission for secret key, Permission = Write
notebook_login()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
#telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts.
from transformers.utils import send_example_telemetry

send_example_telemetry("Question_GEN", framework="pytorch")

##Importing training dataset

In [ ]:
#Importing training datase
raw_datasets = load_dataset("rajpurkar/squad") #squad dataset
metric = load("rouge")
raw_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
#displaying training data set
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(raw_datasets["train"])

,id,title,context,question,answers
0,57316bfea5e9cc1400cdbf3d,Muammar_Gaddafi,"Muammar Gaddafi was born in a tent near Qasr Abu Hadi, a rural area outside the town of Sirte in the deserts of western Libya. His family came from a small, relatively un-influential tribal group called the Qadhadhfa, who were Arabized Berber in heritage. His father, Mohammad Abdul Salam bin Hamed bin Mohammad, was known as Abu Meniar (died 1985), and his mother was named Aisha (died 1978); Abu Meniar earned a meager subsistence as a goat and camel herder. Nomadic Bedouins, they were illiterate and kept no birth records. As such, Gaddafi's date of birth is not known with certainty, and sources have set it in 1942 or in the spring of 1943, although biographers Blundy and Lycett noted that it could have been pre-1940. His parents' only surviving son, he had three older sisters. Gaddafi's upbringing in Bedouin culture influenced his personal tastes for the rest of his life. He repeatedly expressed a preference for the desert over the city and retreated to the desert to meditate.","Other than Mohammad Abdul Salam bin Hamed bin Mohammad, what was Gaddafi's father called?","{'text': ['Abu Meniar'], 'answer_start': [326]}"
1,56d3af3e2ccc5a1400d82e44,Frédéric_Chopin,"Chopin's output as a composer throughout this period declined in quantity year by year. Whereas in 1841 he had written a dozen works, only six were written in 1842 and six shorter pieces in 1843. In 1844 he wrote only the Op. 58 sonata. 1845 saw the completion of three mazurkas (Op. 59). Although these works were more refined than many of his earlier compositions, Zamoyski opines that ""his powers of concentration were failing and his inspiration was beset by anguish, both emotional and intellectual.""",How many works did Chopin write in 1845?,"{'text': ['three mazurkas'], 'answer_start': [264]}"
2,56ce771daab44d1400b887cf,Solar_energy,"Solar chemical processes use solar energy to drive chemical reactions. These processes offset energy that would otherwise come from a fossil fuel source and can also convert solar energy into storable and transportable fuels. Solar induced chemical reactions can be divided into thermochemical or photochemical. A variety of fuels can be produced by artificial photosynthesis. The multielectron catalytic chemistry involved in making carbon-based fuels (such as methanol) from reduction of carbon dioxide is challenging; a feasible alternative is hydrogen production from protons, though use of water as the source of electrons (as plants do) requires mastering the multielectron oxidation of two water molecules to molecular oxygen. Some have envisaged working solar fuel plants in coastal metropolitan areas by 2050 – the splitting of sea water providing hydrogen to be run through adjacent fuel-cell electric power plants and the pure water by-product going directly into the municipal water system. Another vision involves all human structures covering the earth's surface (i.e., roads, vehicles and buildings) doing photosynthesis more efficiently than plants.",What is a possible alternative to making carbon-based fuels from reduction of carbon dioxide?,"{'text': ['hydrogen production from protons'], 'answer_start': [547]}"
3,570a85de6d058f1900182f38,Old_English,"Old English was not static, and its usage covered a period of 700 years, from the Anglo-Saxon settlement of Britain in the 5th century to the late 11th century, some time after the Norman invasion. While indicating that the establishment of dates is an arbitrary process, Albert Baugh dates Old English from 450 to 1150, a period of full inflections, a synthetic language. Perhaps around 85 per cent of Old English words are no longer in use, but those that survived, to be sure, are basic elements of Modern English vocabulary.",In what century did Old English cease to be used?,"{'text': ['11th'], 'answer_start': [147]}"
4,5730bc458ab72b1400f9c748,Sumer,"Commercial credit and agricultural consumer loans were the main typ

In [ ]:
#The metric is an instance of datasets.Metric:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [ ]:
#Initializing model name variable and tokenizer
model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
# pre-set parameters for tokenizing function
prefix = "generate question: "
max_input_length = 1100
max_target_length = 128

# Tokenizing function
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples['context']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples['question'], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


In [ ]:
#displaying samples
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[3806, 822, 10, 30797, 120, 6, 8, 496, 65, 3, 9, 6502, 1848, 5, 71, 2916, 8, 5140, 5450, 31, 7, 2045, 22161, 19, 3, 9, 7069, 12647, 13, 8, 16823, 3790, 5, 3, 29167, 16, 851, 13, 8, 5140, 5450, 11, 5008, 34, 6, 19, 3, 9, 8658, 12647, 13, 2144, 28, 6026, 3, 76, 24266, 28, 8, 9503, 96, 553, 15, 7980, 1980, 1212, 13285, 1496, 1280, 3021, 12, 8, 5140, 5450, 19, 8, 23711, 2617, 13, 8, 3, 24756, 6219, 5, 3, 29167, 1187, 8, 20605, 2617, 19, 8, 8554, 17, 235, 6, 3, 9, 17535, 286, 13, 7029, 11, 9619, 5, 94, 19, 3, 9, 16455, 13, 8, 3, 3844, 17, 235, 44, 301, 1211, 1395, 6, 1410, 213, 8, 16823, 3790, 3, 28285, 26, 120, 4283, 12, 2788, 8942, 9, 26, 1954, 264, 8371, 8283, 16, 507, 3449, 5, 486, 8, 414, 13, 8, 711, 1262, 41, 232, 16, 3, 9, 1223, 689, 24, 1979, 7, 190, 220, 12647, 7, 11, 8, 2540, 10576, 15, 201, 19, 3, 9, 650, 6, 941, 3372, 12647, 13, 3790, 5, 1], [3806, 822, 10, 30797, 120, 6, 8, 496, 65, 3, 9, 6502, 1848, 5, 71, 2916, 8, 5140, 5450, 31, 7, 2045, 22161, 19, 3, 9, 7069,

In [ ]:
#To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier.
#This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

##Fine-tuning a model on Question genration task




In [ ]:
#download the pretrained model and initalizing
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
#declaring Seq2Seq Training Arguments
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-QuestionGen",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    push_to_hub=True,
)

In [ ]:
#data collector
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# Evaluation matrix
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.223600,2.020813,28.890900,9.162300,26.253600,26.248000,13.381100
2,2.154100,1.983846,28.982300,9.226900,26.233900,26.222500,13.865600
3,2.123700,1.973645,28.847200,9.114800,26.114800,26.100500,13.824200


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=16425, training_loss=2.2055503308718607, metrics={'train_runtime': 6678.3468, 'train_samples_per_second': 39.351, 'train_steps_per_second': 2.459, 'total_flos': 2.391066213069619e+16, 'train_loss': 2.2055503308718607, 'epoch': 3.0})

In [ ]:
#pusing trained model to Huggingface
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1715774619.eac26ff3006c.1510.0:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BhuvanGowda/t5-small-finetuned-QuestionGen/commit/3886dc3777fd04c6578f19a1c7cb8cbe817d22b1', commit_message='End of training', commit_description='', oid='3886dc3777fd04c6578f19a1c7cb8cbe817d22b1', pr_url=None, pr_revision=None, pr_num=None)